#YouTube 動画視聴回数予測　コンペ

## データの読み込み

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv("/content/drive/My Drive/遊び/Youtube/train_data.csv")
test = pd.read_csv("/content/drive/My Drive/遊び/Youtube/test_data.csv")

train_x = train.drop(['y'],axis=1)
train_y = np.log1p(train['y']) 

test_x = test.copy()

## データの確認

In [4]:
train.head()

,id,video_id,title,publishedAt,channelId,channelTitle,categoryId,collection_date,tags,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,y
0,1,GDtyztIThRQ,[12] BGM Inazuma Eleven 3 - ~ライオコツト ダンジョン~,2011-01-09T05:50:33.000Z,UCQaNYC3dNvH8FqrEyK7hTJw,DjangoShiny,20,20.01.02,Inazuma|Eleven|Super|Once|bgm|ost|イナズマイレブン|Kyo...,114,0,7,https://i.ytimg.com/vi/GDtyztIThRQ/default.jpg,False,False,~ライオコツト ダンジョン~Inazuma Eleven 3 BGM Complete (R...,29229
1,2,m4H9s3GtTlQ,ねごと - メルシールー [Official Music Video],2012-07-23T03:00:09.000Z,UChMWDi-HBm5aS3jyRSaAWUA,ねごと Official Channel,10,20.08.02,ねごと|ネゴト|メルシールー|Re:myend|リマインド|Lightdentity|ライデ...,2885,50,111,https://i.ytimg.com/vi/m4H9s3GtTlQ/default.jpg,False,False,http://www.negoto.com/全員平成生まれ、蒼山幸子（Vo＆Key)、沙田瑞...,730280
2,3,z19zYZuLuEU,VF3tb 闇よだれvsちび太 (SEGA),2007-07-26T13:54:09.000Z,UCBdcyoZSt5HBLd_n6we-xIg,siropai,24,20.14.01,VF3|VF4|VF5|ちび太|闇よだれ|chibita|virtuafighter|seg...,133,17,14,https://i.ytimg.com/vi/z19zYZuLuEU/default.jpg,False,False,Beat-tribe cup finalhttp://ameblo.jp/siropai/,80667
3,4,pmcIOsL7s98,free frosty weekend!,2005-05-15T02:38:43.000Z,UC7K5am1UAQEsCRhzXpi9i1g,Jones4Carrie,22,19.22.12,frosty,287,51,173,https://i.ytimg.com/vi/pmcIOsL7s98/default.jpg,False,False,I look so bad but look at me!,34826
4,5,ZuQgsTcuM-4,トップ・オブ・ザ・ワールド,2007-09-09T09:52:47.000Z,UCTW1um4R-QWa8iIfITGvlZQ,Tatsuya Maruyama,10,20.08.01,ギター|guitar|南澤大介|トップオブザワールド|トップ|オブ|ワールド|カーペンターズ...,178,6,17,https://i.ytimg.com/vi/ZuQgsTcuM-4/default.jpg,False,False,ソロギターのしらべより「トップオブザワールド」です。クラシックギターで弾いてます。Offic...,172727


In [0]:
print(ps.pivot_table(train,index='',columns=''))

## 前処理

### light_gbm

In [0]:
def LGB_Model(train_x,train_y,test_x,seed):
  rmse_scores = []
  preds = []
  preds_test = []
  va_idxes = []
  pred_cv = np.zeros(len(test_x.index))


  fold_num = 10
  kf = KFold(n_splits=fold_num, shuffle=True, random_state=71)
  for n_fold, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
      tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
      tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
      lgb_train = lgb.Dataset(tr_x,tr_y)
      lgb_eval = lgb.Dataset(va_x,va_y)

      params = {
          'obejective':'rmse',
          'seed':seed,
          'verbose':1,
          'metrics':'rmse',
          'max_depth' : 20,
          #'num_leaves' : 20 # 付け足した5/01
      }

      lgb_model = lgb.train(
          params,
          lgb_train,
          num_boost_round = 500,
          valid_names=['train','valid'],
          valid_sets=[lgb_train,lgb_eval],
          verbose_eval= 100
                    )
      va_pred = lgb_model.predict(va_x)
      mse = mean_squared_error(va_y, va_pred)
      rmse = np.sqrt(mse)
      rmse_scores.append(rmse)
      
      submission = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration) 
      pred_cv += submission/fold_num
      print(n_fold+1,"回目のrmsle:",rmse)

  print("rmsle平均",np.mean(rmse_scores))
  return pred_cv

In [0]:
LGB_Model(train_x,train_y,test_x,seed)

## モデル

## 評価

## 提出